In [ ]:
import os 
import sys 
import requests 
from tqdm import tqdm 



In [ ]:
# Sys.argv là một list trong Python chứa các tham số dòng lệnh với 
# sys.argv[0] là tên của script 

# kiểm tra xem len(sys.argv) != 2 nghĩa là có đúng một tham số được 
# truyền vào cùng với tên script hay không (tổng cộng là 2)
if len(sys.argv) != 2 :
    # nếu không đúng 2 tham số chương trình sẽ in ra thông báo 
    print('You must enter the model name as a parameter, e.g.: download_model.py 117M')
    # và sau đó sử dụng sys.exit(1) để thoát với mã lỗi 1 
    sys.exit(1)

# nếu có đúng hai tham số biến model sẽ được gán giá trị của tham số thứ 2 (
# Tham số đầu tiên nhập vào từ dòng lệnh, không tính tên script)
model = sys.argv[1]

# kiểm tra xem model (script of model) có thuộc 1 trong các phân tử của danh sách hay không 
if model not in ["117M", "PrettyBig", "encoder", "1.5B"]:
    # nếu không đưa ra một thông báo 
    print("Unknown model! Currently available models: 117M, SortaBig, 1.5B")
    # sau đó gọi sys.exit(1) để thoát khỏi chương trình với mã lỗi 1 
    # biểu thị rằng đã có lỗi xảy ra 
    sys.exit(1)

# kiểm tra nếu như không tòn đại 1 đường dẫn tên là script của model 
if not os.path.exists(model):
    # tạo một đường dẫn tên là tên script của model 
    os.makedirs(model)

# tương tự với 1 đường dẫn có tên là encoder 
if not os.path.exists("encoder"):
    os.makedirs("encoder")


# duyệt qua danh sách chứa tên của 2 tệp json và bpe 
for filename in ['encoder.json', 'vocab.bpe']:
    # sử dụng request.get  để gửi 1 HTTP GET request đến URL được chỉ định 
    # URL này kết hợp cơ sở URL với filename để tạo thành 1 URL đầy đủ 
    # mà từ đó dữ liệu sẽ được tải xuống 
    # Tham số này chỉ định rằng nội dung của response sẽ được tải xuống dưới dạng luồng (stream). 
    # Điều này hữu ích khi bạn muốn tải xuống dữ liệu lớn mà không cần phải tải toàn bộ nội dung vào bộ nhớ trước khi xử lý.
    r = requests.get("https://storage.googleapis.com/connors-models/public/encoder/" + filename, stream=True)
    
    # sau đó ta mở file để ghi dữ liệu nhị phân 'wb. Nếu nó không tồn tại 
    # nó sẽ được tạo mới . File này là 1 tệp được được nối từ chuỗi "encoder" với tên "filename"
    with open(os.path.join("encoder", filename), 'wb') as f: 
        # lấy kích thước của file từ headers của HTTP response và chuyển đổi nó thành 
        # 1 số nguyên 
        file_size = int(r.headers["context-length"])
        # Đặt kích thước của mỗi phần dữ liệu là 1000 bytes
        chunk_size = 1000
        # Khởi tạo một thanh tiến chình với chiều rộng cột 100 mô tả "Fetching" cùng với tên 
        # file , tổng kích thước file, và đon vị các bytes 
        with tqdm(ncols=100, desc="Fetching " + filename, total=file_size, unit_scale=True) as pbar:
            # 1k for chunk_size, since Ethernet packet size is around 1500 bytes
            # 1k cho chunk_size vì kích thước của gói ETHENET là khoảng 1500 bytes 
            # chia chunk_size thành các phần có kích thước chunk_Size 
            for chunk in r.iter_content(chunk_size=chunk_size):
                # ghi từng phần dữ liệu từ tệp nguồn F cho file chunk 
                f.write(chunk)
                # Cập nhật thanh tiến trình Process bar mỗi lần ghi dữ liệu '
                pbar.update(chunk_size)

# Kiểm tra xem model có == "encoder"
if model == "encoder":
    # nếu có thì thoát chương trình 
    sys.exit()
# Mở các file model chekcpoint gán cho f 
with open(os.path.join(model, "checkpoint"), "w") as f:
        # ghi một thông tin vào f 
        f.write("model_checkpoint_path: \"model.ckpt\"\n")

for filename in ['model.ckpt.data-00000-of-00001', 'model.ckpt.index', 'model.ckpt.meta']:
    # # sử dụng request.get  để gửi 1 HTTP GET request đến URL được chỉ định 
    # URL này kết hợp cơ sở URL với filename để tạo thành 1 URL đầy đủ 
    # mà từ đó dữ liệu sẽ được tải xuống 
    # Tham số này chỉ định rằng nội dung của response sẽ được tải xuống dưới dạng luồng (stream). 
    # Điều này hữu ích khi bạn muốn tải xuống dữ liệu lớn mà không cần phải tải toàn bộ nội dung vào bộ nhớ trước khi xử lý.
    r = requests.get("https://storage.googleapis.com/connors-models/public/" + model + "/" + filename, stream=True)

    # sau đó ta mở file để ghi dữ liệu nhị phân 'wb. Nếu nó không tồn tại 
    # nó sẽ được tạo mới . File này là 1 tệp được được nối từ chuỗi "encoder" với tên "filename"
    with open(os.path.join(model, filename), 'wb') as f:
        # lấy kích thước của file từ headers của HTTP response và chuyển đổi nó thành 
        # 1 số nguyên 
        file_size = int(r.headers["content-length"])
        chunk_size = 1000
        # Khởi tạo một thanh tiến chình với chiều rộng cột 100 mô tả "Fetching" cùng với tên 
        # file , tổng kích thước file, và đon vị các bytes 
        with tqdm(ncols=100, desc="Fetching " + filename, total=file_size, unit_scale=True) as pbar:
            # 1k for chunk_size, since Ethernet packet size is around 1500 bytes
            for chunk in r.iter_content(chunk_size=chunk_size):
                # ghi từng phần dữ liệu từ tệp nguồn F cho file chunk 
                f.write(chunk)
                pbar.update(chunk_size)
